In [ ]:
import json
import openai

# How to fine-tune a GPT-3 model for specific prompts

I'm constantly looking for ways to automate the work with support requests. An idea has been to fine-tune a GPT-3 model to answer common support-related questions.

**Here's how you can fine-tune a GPT-3 model with Python with your own data.**

In this walkthrough, we'll fine-tune a GPT-3 model to answer common support-related questions.

Detailed step-by-step intructions for this repo in this blog post: https://norahsakal.com/blog/fine-tune-gpt3-model

>### Disclaimer
>This guide walks you through fine-tuning a GPT-3 model in Python, shown in a Jupyter notebook.
>If you're looking for the steps of fine-tuning right in a terminal, [OpenAI has a great guide for fine-tuning in your terminal](https://beta.openai.com/docs/guides/fine-tuning "fine-tuning in terminal").

# Define OpenAI API keys

In [ ]:
api_key ="YOUR_OPENAI_API_KEY"
openai.api_key = api_key

# Create training data

Make sure to end each `prompt` with a suffix. According to the [OpenAI API reference](https://beta.openai.com/docs/guides/fine-tuning "fine-tuning reference"), you can use ` ->`.

Also, make sure to end each `completion` with a suffix as well; I'm using `.\n`.

In [ ]:
data_file = [{
    "prompt": "Prompt ->",
    "completion": " Ideal answer.\n"
},{
    "prompt":"Prompt ->",
    "completion": " Ideal answer.\n"
}]

# Save dict as JSONL

Training data need to be a JSONL document.
JSONL file is a newline-delimited JSON file.
More info about JSONL: https://jsonlines.org/

In [ ]:
file_name = "training_data.jsonl"

with open(file_name, 'w') as outfile:
    for entry in data_file:
        json.dump(entry, outfile)
        outfile.write('\n')

# Check JSONL file

In [ ]:
!openai tools fine_tunes.prepare_data -f training_data.jsonl

# Upload file to your OpenAI account

In [ ]:
upload_response = openai.File.create(
  file=open(file_name, "rb"),
  purpose='fine-tune'
)
upload_response

# Save file name

In [ ]:
file_id = upload_response.id
file_id

# Fine-tune a model

The default model is **Curie**. 

If you'd like to use **DaVinci** instead, then add it as a base model to fine-tune:

```openai.FineTune.create(training_file=file_id, model="davinci")```

In [ ]:
fine_tune_response = openai.FineTune.create(training_file=file_id)
fine_tune_response

# Check latest fine-tuning event

In [ ]:
fine_tune_response.events[-1]

# Save fine-tuned model

In [ ]:
fine_tuned_model = fine_tune_response.fine_tuned_model
fine_tuned_model

# Test the new model on a new prompt

Remember to end the prompt with the same suffix as we used in the training data; ` ->`:

In [ ]:
new_prompt = "NEW PROMPT ->"

In [ ]:
answer = openai.Completion.create(
  model=fine_tuned_model,
  prompt=new_prompt,
  max_tokens=10, # Change amount of tokens for longer completion
  temperature=0
)
answer['choices'][0]['text']